In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Sun May 29 00:06:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 34%   44C    P8    39W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 29%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    
    # db_cn_5 = db.w_patent_cn_5
    # db_cn_10 = db.w_patent_cn_10
    # db_us_5 = db.w_patent_us_5
    # db_us_10 = db.w_patent_us_10
    # db_de_5 = db.w_patent_de_5
    # db_de_10 = db.w_patent_de_10
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_rm_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_de')

# 使用已去除重複priority的資料依年份切資料

## 2010

### CN

In [7]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2010-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [8]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [9]:
# 存進MongoDB
db.rm_patent_cn_2010.insert_many(db_cn_2010.to_dict('records'))

### DE

In [15]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [12]:
# 存進MongoDB
db.rm_patent_de_2010.insert_many(db_de_2010.to_dict('records'))

TypeError: documents must be a non-empty list

### US

In [17]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [18]:
db_us_2010

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009c0f7b73c16636b3109a,US20130223235A1,"[Rose Qingyang Hu, Zhijun Cai, Chandra Sekhar ...",[BlackBerry Ltd],[Research in Motion Ltd],2010-07-14 04:00:00,20130829,N/A,"[H04W36/30, H04W48/20, H04W36/04, H04W84/045]",A UE comprising a processor configured to perf...,IdleModeHybridMobilityProcedureinaHeterogeneou...,This application is a filing under 35 U.S.C. 3...,[1.AUEcomprising:aprocessorconfiguredtoperform...,"[US, EP, JP, KR, CN, CA, IN, TW, WO]","[{'pto': 'US', 'patentNumber': '20070123252', ...","[{'pto': 'GB', 'patentNumber': '0413534', 'kin...","[{'pto': 'US', 'patentNumber': '20120276933', ...","[{'pto': 'KR', 'patentNumber': '2012003490', '...","[{'publication': 'US20130223235A1', 'time': '2...",PCT/US2010/042018
1,6200a1987b73c16636b310d8,US8458315B2,"[Markus Miche, Oliver Baecker, Tobias Bauer, D...",[SAP SE],[SAP SE ],2010-12-06 05:00:00,20130604,20130515,"[H04W4/00, H04L67/12, H04W4/029, H04L67/2857, ...",Embodiments of the present invention include s...,In-vehicleapplicationplatformforvehicle-to-bus...,"The present invention relates to computing, an...",[1.Acomputer-implementedmethodcomprisingoperat...,"[US, EP]","[{'pto': 'US', 'patentNumber': '7382289', 'kin...",None,"[{'pto': 'US', 'patentNumber': '9098312', 'kin...","[{'pto': 'DE', 'patentNumber': '102011107111',...","[{'publication': 'US8458315B2', 'time': '2013-...","US12/961,300"
2,6200b2f77b73c16636b31110,US10198333B2,"[Mark B. Trobough, Keshavan K. Tiruvallur, Chi...",[Intel Corp],[Intel Corp],2010-12-23 05:00:00,20190205,20190116,"[G06F11/36, G06F11/2733, G06F11/22, G06F11/267...",An apparatus and method is described herein fo...,"Test,validation,anddebugarchitecture",This patent application is a U.S. National Pha...,[1.Anapparatuscomprising:atleastonehardwaretes...,"[US, JP, KR, CN, DE, GB, WO]","[{'pto': 'JP', 'patentNumber': 'H0831559', 'ki...","[{'pto': 'JP', 'patentNumber': '200108561', 'k...","[{'pto': 'US', 'patentNumber': '20180348301', ...","[{'pto': 'US', 'patentNumber': '7895565', 'kin...","[{'publication': 'US10198333B2', 'time': '2019...",PCT/US2010/061995
3,6200c24e7b73c16636b311ac,US9030941B2,"[Juergen Michel, Clemens Suerbaum]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],2010-06-16 04:00:00,20150512,20150422,"[H04W52/0206, H04W24/02, H04W24/08, H04W88/08,...",There is proposed a mechanism for power saving...,Powersavingprocedureincommunicationsnetwork,1. Field of the InventionThe present invention...,[1.Apparatuscomprising:memorycomprisingcompute...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6259681', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20140105056', ...","[{'pto': 'US', 'patentNumber': '20110312359', ...","[{'publication': 'US9030941B2', 'time': '2015-...",PCT/EP2010/058450
4,6200c3ae7b73c16636b311d2,US9198185B2,"[Istvan Zsolt Kovacs, Frank Frederiksen, Klaus...",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],2010-02-10 05:00:00,20151124,20151104,"[H04W72/082, H04B1/707, H04L5/0007, H04L5/001,...",There is proposed a mechanism by means of whic...,Mechanismforaggregatinguplinkinterferenceinfor...,1. Field of the InventionThe present invention...,[1.Amethodcomprising:monitoringuplinkanddownli...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '20080247375', ...",None,"[{'pto': 'CN', 'patentNumber': '11213163', 'ki...","[{'pto': 'EP', 'patentNumber': '2534866', 'kin...","[{'publication': 'US9198185B2', 'time': '2015-...",PCT/EP2010/051662
5,6200c56e7b73c16636b311fa,US8891436B2,"[Jianqiang Zhang, Mingxi Fan, Jiming Guo, Bo C...",[Qualcomm Inc],[Qualcomm Inc],2010-10-12 04:00:00,20141118,20141029,"[H04B7/216, H04W52/365, H04W72/14, H04W80/04, ...",A method and apparatus for effectively facilit...,Methodandapparatusforfacilitatingeffectivesch

In [19]:
# 存進MongoDB
db.rm_patent_us_2010.insert_many(db_us_2010.to_dict('records'))

## 2011

### CN

In [20]:
dateStr = "2011-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2011-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [21]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [22]:
# 存進MongoDB
db.rm_patent_cn_2011.insert_many(db_cn_2010.to_dict('records'))

### DE

In [23]:
dateStr = "2011-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2011-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [24]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [25]:
# 存進MongoDB
db.rm_patent_de_2011.insert_many(db_de_2010.to_dict('records'))

TypeError: documents must be a non-empty list

### US

In [26]:
dateStr = "2011-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2011-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [27]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [28]:
# 存進MongoDB
db.rm_patent_us_2011.insert_many(db_us_2010.to_dict('records'))

## 2012

### CN

In [29]:
dateStr = "2012-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2012-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [30]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [31]:
# 存進MongoDB
db.rm_patent_cn_2012.insert_many(db_cn_2010.to_dict('records'))

### DE

In [32]:
dateStr = "2012-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2012-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [33]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [34]:
# 存進MongoDB
db.rm_patent_de_2012.insert_many(db_de_2010.to_dict('records'))

### US

In [35]:
dateStr = "2012-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2012-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [36]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [37]:
# 存進MongoDB
db.rm_patent_us_2012.insert_many(db_us_2010.to_dict('records'))

## 2013

### CN

In [38]:
dateStr = "2013-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2013-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [39]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [40]:
# 存進MongoDB
db.rm_patent_cn_2013.insert_many(db_cn_2010.to_dict('records'))

### DE

In [41]:
dateStr = "2013-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2013-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [42]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [43]:
# 存進MongoDB
db.rm_patent_de_2013.insert_many(db_de_2010.to_dict('records'))

### US

In [44]:
dateStr = "2013-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2013-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [45]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [46]:
# 存進MongoDB
db.rm_patent_us_2013.insert_many(db_us_2010.to_dict('records'))

## 2014

### CN

In [47]:
dateStr = "2014-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2014-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [48]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [49]:
# 存進MongoDB
db.rm_patent_cn_2014.insert_many(db_cn_2010.to_dict('records'))

### DE

In [50]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [51]:
# 存進MongoDB
db.rm_patent_de_2014.insert_many(db_de_2010.to_dict('records'))

### US

In [52]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [53]:
# 存進MongoDB
db.rm_patent_us_2014.insert_many(db_us_2010.to_dict('records'))

## 2015

### CN

In [54]:
dateStr = "2015-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [55]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [56]:
# 存進MongoDB
db.rm_patent_cn_2015.insert_many(db_cn_2010.to_dict('records'))

### DE

In [57]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [58]:
# 存進MongoDB
db.rm_patent_de_2015.insert_many(db_de_2010.to_dict('records'))

### US

In [59]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [60]:
# 存進MongoDB
db.rm_patent_us_2015.insert_many(db_us_2010.to_dict('records'))

## 2016

### CN

In [61]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2016-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [62]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [63]:
# 存進MongoDB
db.rm_patent_cn_2016.insert_many(db_cn_2010.to_dict('records'))

### DE

In [64]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [65]:
# 存進MongoDB
db.rm_patent_de_2016.insert_many(db_de_2010.to_dict('records'))

### US

In [66]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [67]:
# 存進MongoDB
db.rm_patent_us_2016.insert_many(db_us_2010.to_dict('records'))

## 2017

### CN

In [68]:
dateStr = "2017-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2017-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [69]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [70]:
# 存進MongoDB
db.rm_patent_cn_2017.insert_many(db_cn_2010.to_dict('records'))

### DE

In [71]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [72]:
# 存進MongoDB
db.rm_patent_de_2017.insert_many(db_de_2010.to_dict('records'))

### US

In [73]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [74]:
# 存進MongoDB
db.rm_patent_us_2017.insert_many(db_us_2010.to_dict('records'))

## 2018

### CN

In [75]:
dateStr = "2018-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2018-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [76]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [77]:
# 存進MongoDB
db.rm_patent_cn_2018.insert_many(db_cn_2010.to_dict('records'))

### DE

In [78]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [79]:
# 存進MongoDB
db.rm_patent_de_2018.insert_many(db_de_2010.to_dict('records'))

### US

In [80]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [81]:
# 存進MongoDB
db.rm_patent_us_2018.insert_many(db_us_2010.to_dict('records'))

## 2019

### CN

In [82]:
dateStr = "2019-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2019-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [83]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [84]:
# 存進MongoDB
db.rm_patent_cn_2019.insert_many(db_cn_2010.to_dict('records'))

### DE

In [85]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [86]:
# 存進MongoDB
db.rm_patent_de_2019.insert_many(db_de_2010.to_dict('records'))

### US

In [87]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [88]:
# 存進MongoDB
db.rm_patent_us_2019.insert_many(db_us_2010.to_dict('records'))

## 2020

### CN

In [89]:
dateStr = "2020-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [90]:
db_cn_2010 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [91]:
# 存進MongoDB
db.rm_patent_cn_2020.insert_many(db_cn_2010.to_dict('records'))

### DE

In [92]:
db_de_2010 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [93]:
# 存進MongoDB
db.rm_patent_de_2020.insert_many(db_de_2010.to_dict('records'))

### US

In [94]:
db_us_2010 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [95]:
# 存進MongoDB
db.rm_patent_us_2020.insert_many(db_us_2010.to_dict('records'))